In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [ ]:
num_samples = 10000
generate_past_months = 60 # for loop
generate_months_for_ds = 49 #  while loop, min 13
dates = pd.date_range(start='2022-02-01', end='2025-04-01', freq='ME')[::-1]

In [ ]:
# Parameters
sav_mean = 3000
sav_std_dev = 900
sav_sav_min_val = 30
sav_max_val = 1000000
sav_num_samples = num_samples
# months = 12

np.random.seed(42)  #| For reproducibility
samples = np.random.normal(loc=sav_mean, scale=sav_std_dev, size=sav_num_samples).round(2)

df = pd.DataFrame({'sav_bal': samples})


# Parameters
rrsp_mean = 20000
rrsp_std_dev = 5000
rrsp_min_val = 0
rrsp_max_val = 10000000
rrsp_num_samples = num_samples

rrspsamples = np.random.normal(loc=rrsp_mean, scale=rrsp_std_dev, size=rrsp_num_samples).round()

df['rrsp_bal'] = rrspsamples

for month in range(1, generate_past_months):
  if month == 1:
    df[f'sav_bal_m_{month}'] = df['sav_bal'] + np.random.normal(loc=0, scale=500, size=len(df)).round(2)
    df[f'rrsp_bal_m_{month}'] = df['rrsp_bal'] * 1.02 + np.random.normal(loc=0, scale=100, size=len(df)).round(2)
  else:
    df[f'sav_bal_m_{month}'] = df[f'sav_bal_m_{month-1}'] + np.random.normal(loc=0, scale=500, size=len(df)).round(2)
    df[f'rrsp_bal_m_{month}'] = df[f'rrsp_bal_m_{month-1}'] * 1.02 + np.random.normal(loc=0, scale=100, size=len(df)).round(2)

completed_df = pd.DataFrame()
temp_df = pd.DataFrame()
nested_dict = {}
start = 1
end = 13
half = 7

while end <= generate_months_for_ds:
  monthly_cols_full = [f'sav_bal_m_{i}' for i in range(start, end)]
  monthly_cols_half = [f'sav_bal_m_{i}' for i in range(start, half)]
  print(monthly_cols_half)
  temp_df['avg_sav_bal_6_mean'] = df[monthly_cols_half].apply(lambda row: row.mean(), axis=1).round(2)
  temp_df['avg_sav_bal_12_mean'] = df[monthly_cols_full].apply(lambda row: row.mean(), axis=1).round(2)

  temp_df['sav_bal_cv_6_month'] = (df[monthly_cols_half].std(axis=1) / df[monthly_cols_half].mean(axis=1) * 100).round(2)
  temp_df['sav_bal_cv_12_month'] = (df[monthly_cols_full].std(axis=1) / df[monthly_cols_full].mean(axis=1) * 100).round(2)

  temp_df['sav_bal_6_month_max'] = df[monthly_cols_half].max(axis=1)
  temp_df['sav_bal_12_month_max'] = df[monthly_cols_full].max(axis=1)

  temp_df['sav_bal_6_month_min'] = df[monthly_cols_half].min(axis=1)
  temp_df['sav_bal_12_month_min'] = df[monthly_cols_full].min(axis=1)

  temp_df['sav_bal_cut_off'] = df['sav_bal'].describe()['75%']
  temp_df['high_sav_bal'] = np.where(temp_df['avg_sav_bal_12_mean'] > temp_df['sav_bal_cut_off'], 1, 0)

  temp_df['sav_bal_std_12_month'] = df[monthly_cols_full].std(axis=1)
  temp_df['sav_bal_std_6_month'] = df[monthly_cols_half].std(axis=1)

  y_6m_sav = df[monthly_cols_half].values
  y_12m_sav = df[monthly_cols_full].values
  temp_df[['sav_bal_slope_6m', 'sav_bal_intercept_6m']] = pd.DataFrame(np.apply_along_axis(lambda row: np.polyfit(np.arange(1, 7), row, deg=1), axis=1, arr=y_6m_sav))
  temp_df[['sav_bal_slope_12m', 'sav_bal_intercept_12m']] = pd.DataFrame(np.apply_along_axis(lambda row: np.polyfit(np.arange(1, 13), row, deg=1),axis=1, arr=y_12m_sav))

  # RRSP
  monthly_cols_full_rrsp = [f'rrsp_bal_m_{i}' for i in range(start, end)]
  monthly_cols_half_rrsp = [f'rrsp_bal_m_{i}' for i in range(start, half)]

  temp_df['avg_rrsp_bal_6_mean'] = df[monthly_cols_half_rrsp].apply(lambda row: row.mean(), axis=1).round(2)
  temp_df['avg_rrsp_bal_12_mean'] = df[monthly_cols_full_rrsp].apply(lambda row: row.mean(), axis=1).round(2)

  temp_df['rrsp_bal_cv_6_month'] = (df[monthly_cols_half_rrsp].std(axis=1) / df[monthly_cols_half_rrsp].mean(axis=1) * 100).round(2)
  temp_df['rrsp_bal_cv_12_month'] = (df[monthly_cols_full_rrsp].std(axis=1) / df[monthly_cols_full_rrsp].mean(axis=1) * 100).round(2)

  temp_df['rrsp_bal_6_month_max'] = df[monthly_cols_half_rrsp].max(axis=1)
  temp_df['rrsp_bal_12_month_max'] = df[monthly_cols_full_rrsp].max(axis=1)

  temp_df['rrsp_bal_6_month_min'] = df[monthly_cols_half_rrsp].min(axis=1)
  temp_df['rrsp_bal_12_month_min'] = df[monthly_cols_full_rrsp].min(axis=1)

  temp_df['rrsp_bal_std_6_month'] = df[monthly_cols_half_rrsp].std(axis=1)
  temp_df['rrsp_bal_std_12_month'] = df[monthly_cols_full_rrsp].std(axis=1)

  y_6m_rrsp = df[monthly_cols_half_rrsp].values
  y_12m_rrsp = df[monthly_cols_full_rrsp].values
  temp_df[['rrsp_bal_slope_6m', 'rrsp_bal_intercept_6m']] = pd.DataFrame(np.apply_along_axis(lambda row: np.polyfit(np.arange(1, 7), row, deg=1), axis=1, arr=y_6m_rrsp))
  temp_df[['rrsp_bal_slope_12m', 'rrsp_bal_intercept_12m']] = pd.DataFrame(np.apply_along_axis(lambda row: np.polyfit(np.arange(1, 13), row, deg=1),axis=1, arr=y_12m_rrsp))


  nested_dict[dates[1]] = {
        # sav
        'sav_bal': df['sav_bal'],
        'avg_sav_bal_6_mean': temp_df['avg_sav_bal_6_mean'],
        'avg_sav_bal_12_mean': temp_df['avg_sav_bal_12_mean'],
        'sav_bal_cv_6_month': temp_df['sav_bal_cv_6_month'],
        'sav_bal_cv_12_month': temp_df['sav_bal_cv_12_month'],

        'sav_bal_6_month_max': temp_df['sav_bal_6_month_max'],
        'sav_bal_12_month_max': temp_df['sav_bal_12_month_max'],

        'sav_bal_6_month_min': temp_df['sav_bal_6_month_min'],
        'sav_bal_12_month_min': temp_df['sav_bal_12_month_min'],

        'high_sav_bal': temp_df['high_sav_bal'],

        'sav_bal_std_6_month': temp_df['sav_bal_std_6_month'],
        'sav_bal_std_12_month': temp_df['sav_bal_std_12_month'],

        'sav_bal_slope_6m': temp_df['sav_bal_slope_6m'],
        'sav_bal_slope_12m': temp_df['sav_bal_slope_12m'],

        # RRSP
        'rrsp_bal': df['rrsp_bal'],
        'avg_rrsp_bal_6_mean': temp_df['avg_rrsp_bal_6_mean'],
        'avg_rrsp_bal_12_mean': temp_df['avg_rrsp_bal_12_mean'],
        'rrsp_bal_cv_6_month': temp_df['rrsp_bal_cv_6_month'],
        'rrsp_bal_cv_12_month': temp_df['rrsp_bal_cv_12_month'],

        'rrsp_bal_6_month_max': temp_df['rrsp_bal_6_month_max'],
        'rrsp_bal_12_month_max': temp_df['rrsp_bal_12_month_max'],

        'rrsp_bal_6_month_min': temp_df['rrsp_bal_6_month_min'],
        'rrsp_bal_12_month_min': temp_df['rrsp_bal_12_month_min'],

        'high_sav_bal': temp_df['high_sav_bal'],

        'rrsp_bal_6_month_min': temp_df['rrsp_bal_6_month_min'],
        'rrsp_bal_12_month_min': temp_df['rrsp_bal_12_month_min'],

        'rrsp_bal_std_6_month': temp_df['rrsp_bal_std_6_month'],
        'rrsp_bal_std_12_month': temp_df['rrsp_bal_std_12_month'],

        'rrsp_bal_slope_6m': temp_df['rrsp_bal_slope_6m'],
        'rrsp_bal_slope_12m': temp_df['rrsp_bal_slope_12m'],


  }

  list_date = ([str(dates[start])] * num_samples)
  print(list_date[0])
  date_df = pd.DataFrame({'date':list_date})
  temp_df = pd.concat([date_df, pd.DataFrame.from_dict(nested_dict[dates[1]])], axis=1)
  completed_df = pd.concat([completed_df, temp_df], axis=0)
  nested_dict.clear()
  start += 1
  end += 1
  half += 1

['sav_bal_m_1', 'sav_bal_m_2', 'sav_bal_m_3', 'sav_bal_m_4', 'sav_bal_m_5', 'sav_bal_m_6']
2025-02-28 00:00:00
['sav_bal_m_2', 'sav_bal_m_3', 'sav_bal_m_4', 'sav_bal_m_5', 'sav_bal_m_6', 'sav_bal_m_7']
2025-01-31 00:00:00
['sav_bal_m_3', 'sav_bal_m_4', 'sav_bal_m_5', 'sav_bal_m_6', 'sav_bal_m_7', 'sav_bal_m_8']
2024-12-31 00:00:00
['sav_bal_m_4', 'sav_bal_m_5', 'sav_bal_m_6', 'sav_bal_m_7', 'sav_bal_m_8', 'sav_bal_m_9']
2024-11-30 00:00:00
['sav_bal_m_5', 'sav_bal_m_6', 'sav_bal_m_7', 'sav_bal_m_8', 'sav_bal_m_9', 'sav_bal_m_10']
2024-10-31 00:00:00
['sav_bal_m_6', 'sav_bal_m_7', 'sav_bal_m_8', 'sav_bal_m_9', 'sav_bal_m_10', 'sav_bal_m_11']
2024-09-30 00:00:00
['sav_bal_m_7', 'sav_bal_m_8', 'sav_bal_m_9', 'sav_bal_m_10', 'sav_bal_m_11', 'sav_bal_m_12']
2024-08-31 00:00:00
['sav_bal_m_8', 'sav_bal_m_9', 'sav_bal_m_10', 'sav_bal_m_11', 'sav_bal_m_12', 'sav_bal_m_13']
2024-07-31 00:00:00
['sav_bal_m_9', 'sav_bal_m_10', 'sav_bal_m_11', 'sav_bal_m_12', 'sav_bal_m_13', 'sav_bal_m_14']
2024-0

In [ ]:
df[monthly_cols_full]

,sav_bal_m_37,sav_bal_m_38,sav_bal_m_39,sav_bal_m_40,sav_bal_m_41,sav_bal_m_42,sav_bal_m_43,sav_bal_m_44,sav_bal_m_45,sav_bal_m_46,sav_bal_m_47,sav_bal_m_48
0,8568.22,8535.47,8095.48,8433.69,8988.43,8711.57,8558.00,7674.40,8981.08,8956.11,9330.54,9395.75
1,3376.32,3132.51,3440.51,3700.38,4096.83,4385.70,4860.59,4710.54,4794.65,4321.20,3967.47,3362.30
2,5934.63,7226.01,7380.74,7632.03,8497.53,8579.93,8500.49,8715.52,7816.32,8035.78,8192.09,7860.44
3,-257.02,-508.40,211.36,117.80,850.19,815.72,1331.99,204.93,152.93,242.03,-142.97,-413.98
4,5701.54,6722.42,7906.37,8076.19,8267.83,8068.70,8381.05,8579.17,8891.56,9347.07,8689.30,8045.96
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2339.18,2405.42,2113.28,853.15,1174.10,862.26,1103.75,1534.07,2093.81,2612.70,1962.21,1903.23
9996,2460.75,2499.87,2614.37,3757.12,3730.97,3993.68,3775.90,3566.15,3114.05,1714.98,1783.79,2403.77
9997,2927.77,3542.11,3823.91,3482.65,4516.23,3866.49,3847.90,3722.28,3391.19,3693.03,4457.89,4251.69
9998,2386.85,2255.96,3186.89,3805.65,4391.81,3706.68,3355.68,3163.23,3620.44,3229.89,2743.52,2220.75


In [ ]:
df[monthly_cols_half]

,sav_bal_m_37,sav_bal_m_38,sav_bal_m_39,sav_bal_m_40,sav_bal_m_41,sav_bal_m_42
0,8568.22,8535.47,8095.48,8433.69,8988.43,8711.57
1,3376.32,3132.51,3440.51,3700.38,4096.83,4385.70
2,5934.63,7226.01,7380.74,7632.03,8497.53,8579.93
3,-257.02,-508.40,211.36,117.80,850.19,815.72
4,5701.54,6722.42,7906.37,8076.19,8267.83,8068.70
...,...,...,...,...,...,...
9995,2339.18,2405.42,2113.28,853.15,1174.10,862.26
9996,2460.75,2499.87,2614.37,3757.12,3730.97,3993.68
9997,2927.77,3542.11,3823.91,3482.65,4516.23,3866.49
9998,2386.85,2255.96,3186.89,3805.65,4391.81,3706.68


In [ ]:
completed_df

,date,sav_bal,avg_sav_bal_6_mean,avg_sav_bal_12_mean,sav_bal_cv_6_month,sav_bal_cv_12_month,sav_bal_6_month_max,sav_bal_12_month_max,sav_bal_6_month_min,sav_bal_12_month_min,...,rrsp_bal_cv_6_month,rrsp_bal_cv_12_month,rrsp_bal_6_month_max,rrsp_bal_12_month_max,rrsp_bal_6_month_min,rrsp_bal_12_month_min,rrsp_bal_std_6_month,rrsp_bal_std_12_month,rrsp_bal_slope_6m,rrsp_bal_slope_12m
0,2022-02-28 00:00:00,3447.04,3770.40,4369.76,9.89,17.75,4271.97,5733.49,3385.25,3385.25,...,4.42,7.67,18822.654415,21104.497416,16742.100000,16742.100000,782.828031,1452.618057,417.710235,402.233316
1,2022-02-28 00:00:00,2875.56,3392.13,3182.31,9.42,12.42,3805.02,3805.02,3017.22,2366.57,...,3.88,7.94,20784.644009,23935.496665,18736.960000,18736.960000,766.076516,1678.006237,409.193669,463.570411
2,2022-02-28 00:00:00,3582.92,3286.31,3566.91,8.89,14.50,3808.12,4801.36,3015.57,3015.57,...,3.30,6.42,18970.489977,21094.747536,17294.560000,17294.560000,598.947562,1229.636617,318.744665,340.595924
3,2022-02-28 00:00:00,4370.73,4026.61,3225.34,15.29,30.83,4705.80,4705.80,3077.85,1660.91,...,4.33,8.12,23505.597608,27064.823093,20978.010000,20978.010000,966.602235,1944.448098,514.588333,538.645037
4,2022-02-28 00:00:00,2789.26,2363.41,2531.85,7.51,11.95,2566.84,3166.86,2044.22,2044.22,...,3.09,6.57,29025.939640,32525.270565,26608.140000,26608.140000,861.374079,1936.631767,457.055592,535.485678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2025-02-28 00:00:00,4170.99,1624.57,1746.43,45.55,35.50,2405.42,2612.70,853.15,853.15,...,3.96,7.57,51450.101688,58271.302170,46312.247549,46312.247549,1929.848901,3934.626801,1027.606070,1090.117501
9996,2025-02-28 00:00:00,1201.49,3176.13,2951.28,22.70,27.37,3993.68,3993.68,2460.75,1714.98,...,3.90,7.19,61884.879415,69393.579200,55734.460589,55734.460589,2292.780489,4487.844857,1224.596685,1244.485457
9997,2025-02-28 00:00:00,2365.21,3693.19,3793.60,14.21,11.97,4516.23,4516.23,2927.77,2927.77,...,3.86,7.14,49689.984563,55788.418441,44784.581493,44784.581493,1827.361205,3588.301647,976.222230,994.984727
9998,2025-02-28 00:00:00,3446.19,3288.97,3172.28,25.62,21.16,4391.81,4391.81,2255.96,2220.75,...,3.67,7.12,64761.907412,73006.897083,58622.003797,58622.003797,2266.525357,4673.981248,1210.997233,1295.505581


In [ ]:
# Parameters
c_mean = 3
c_std_dev = 1
c_min_val = 0
c_max_val = 30
c_num_samples = num_samples

c_samples = np.random.normal(loc=c_mean, scale=c_std_dev, size=c_num_samples).round()


df = pd.DataFrame({'calls_to_branch': c_samples})

# Parameters
v_mean = 5
v_std_dev = 2
v_min_val = 0
v_max_val = 30
v_num_samples = num_samples

v_samples = np.random.normal(loc=v_mean, scale=v_std_dev, size=v_num_samples).round()

df['visit_to_branch'] = v_samples

# Parameters
s_mean = 30
s_std_dev = 30
s_min_val = 0
s_max_val = 2000
s_num_samples = num_samples
s_months = 12
df['so_eng'] = np.random.normal(loc=s_mean, scale=s_std_dev, size=s_num_samples).round() # np.random.choice(range(0, 2), df.shape[0])

completed_df = pd.DataFrame()
temp_df = pd.DataFrame()
nested_dict = {}
start = 1
end = 13
half = 7

for month in range(1, generate_past_months):
  if month == 1:
    df[f'calls_to_branch_m_{month}'] = df['calls_to_branch'] + np.random.normal(loc=0, scale=1, size=len(df)).round()
    df[f'calls_to_branch_m_{month}'] = np.where(df[f'calls_to_branch_m_{month}'] < 0, 0, df[f'calls_to_branch_m_{month}'])

    df[f'visit_to_branch_m_{month}'] = df['visit_to_branch'] + np.random.normal(loc=0, scale=1, size=len(df)).round()
    df[f'visit_to_branch_m_{month}'] = np.where(df[f'visit_to_branch_m_{month}'] < 0, 0, df[f'visit_to_branch_m_{month}'])

    df[f'so_eng_m_{month}_count'] = df['so_eng'] + np.random.normal(loc=0, scale=5, size=len(df)).round()
    df[f'so_eng_m_{month}_count'] = np.where(df[f'so_eng_m_{month}_count'] < 0, 0, df[f'so_eng_m_{month}_count'])

  else:
    df[f'calls_to_branch_m_{month}'] = df[f'calls_to_branch_m_{month-1}'] + np.random.normal(loc=0, scale=1, size=len(df)).round()
    df[f'calls_to_branch_m_{month}'] = np.where(df[f'calls_to_branch_m_{month}'] < 0, 0, df[f'calls_to_branch_m_{month}'])

    df[f'visit_to_branch_m_{month}'] = df[f'visit_to_branch_m_{month-1}'] + np.random.normal(loc=0, scale=1, size=len(df)).round()
    df[f'visit_to_branch_m_{month}'] = np.where(df[f'visit_to_branch_m_{month}'] < 0, 0, df[f'visit_to_branch_m_{month}'])

    df[f'so_eng_m_{month}_count'] =  df[f'so_eng_m_{month-1}_count'] + np.random.normal(loc=0, scale=5, size=len(df)).round()
    df[f'so_eng_m_{month}_count'] = np.where(df[f'so_eng_m_{month}_count'] < 0, 0, df[f'so_eng_m_{month}_count'])

while end <= generate_months_for_ds:
  print(start, end, half)
  monthly_ctb_cols12 = [f'calls_to_branch_m_{i}' for i in range(start, end)]
  temp_df['call_to_branch_std_12m'] = df[monthly_ctb_cols12].std(axis=1)

  monthly_ctb_cols6 = [f'calls_to_branch_m_{i}' for i in range(start, half)]
  temp_df['call_to_branch_std_6m'] = df[monthly_ctb_cols6].std(axis=1)

  monthly_vtb_cols12 = [f'visit_to_branch_m_{i}' for i in range(1, 13)]
  temp_df['visit_to_branch_std_12m'] = df[monthly_vtb_cols12].std(axis=1)

  monthly_vtb_cols6 = [f'visit_to_branch_m_{i}' for i in range(1, 7)]
  temp_df['visit_to_branch_std_6m'] = df[monthly_vtb_cols6].std(axis=1)

  temp_df['total_to_branch_std_12m'] =  df[monthly_ctb_cols12 + monthly_vtb_cols12].std(axis=1)
  temp_df['total_to_branch_std_6m'] = df[monthly_ctb_cols6 + monthly_vtb_cols6].std(axis=1)

  temp_df['so_eng_1m_ago'] = np.where(df['so_eng_m_1_count'] > 0, 1, 0)
  temp_df['so_eng_3m_ago'] = np.where(df['so_eng_m_3_count'] > 0, 1, 0)


  nested_dict[dates[1]] = {
        'call_to_branch_std_6m': temp_df['call_to_branch_std_6m'],
        'call_to_branch_std_12m': temp_df['call_to_branch_std_12m'],

        'visit_to_branch_std_6m': temp_df['visit_to_branch_std_6m'],
        'visit_to_branch_std_12m': temp_df['visit_to_branch_std_12m'],

        'total_to_branch_std_6m': temp_df['total_to_branch_std_6m'],
        'total_to_branch_std_12m': temp_df['total_to_branch_std_12m'],

        'so_eng_1m_ago': temp_df['so_eng_1m_ago'],
        'so_eng_3m_ago': temp_df['so_eng_3m_ago'],
        'so_eng_1m_ago_count': df['so_eng_m_1_count'],
        'so_eng_3m_ago_count': df['so_eng_m_3_count'],
  }

  list_date = ([str(dates[start])] * num_samples)
  print(list_date[0])
  date_df = pd.DataFrame({'date':list_date})
  temp1_df = pd.concat([date_df, pd.DataFrame.from_dict(nested_dict[dates[1]])], axis=1)
  completed_df = pd.concat([completed_df, temp1_df], axis=0)
  nested_dict.clear()
  start += 1
  end += 1
  half += 1

completed_df['small_bus_owner_flag'] = np.random.choice(range(0, 2), completed_df.shape[0])
completed_df['std_prod_own_flag'] = np.random.choice(range(0, 2), completed_df.shape[0])
completed_df['tfsa_prod_own_12m'] = np.random.choice(range(0, 2), completed_df.shape[0])
completed_df['tranact_prd_count'] = np.random.choice(range(0, 2), completed_df.shape[0])
completed_df['unsecure_loc_count_flag'] = np.random.choice(range(0, 2), completed_df.shape[0])

1 13 7
2025-02-28 00:00:00
2 14 8
2025-01-31 00:00:00
3 15 9
2024-12-31 00:00:00
4 16 10
2024-11-30 00:00:00
5 17 11
2024-10-31 00:00:00
6 18 12
2024-09-30 00:00:00
7 19 13
2024-08-31 00:00:00
8 20 14
2024-07-31 00:00:00
9 21 15
2024-06-30 00:00:00
10 22 16
2024-05-31 00:00:00
11 23 17
2024-04-30 00:00:00
12 24 18
2024-03-31 00:00:00
13 25 19
2024-02-29 00:00:00
14 26 20
2024-01-31 00:00:00
15 27 21
2023-12-31 00:00:00
16 28 22
2023-11-30 00:00:00
17 29 23
2023-10-31 00:00:00
18 30 24
2023-09-30 00:00:00
19 31 25
2023-08-31 00:00:00
20 32 26
2023-07-31 00:00:00
21 33 27
2023-06-30 00:00:00
22 34 28
2023-05-31 00:00:00
23 35 29
2023-04-30 00:00:00
24 36 30
2023-03-31 00:00:00
25 37 31
2023-02-28 00:00:00
26 38 32
2023-01-31 00:00:00
27 39 33
2022-12-31 00:00:00
28 40 34
2022-11-30 00:00:00
29 41 35
2022-10-31 00:00:00
30 42 36
2022-09-30 00:00:00
31 43 37
2022-08-31 00:00:00
32 44 38
2022-07-31 00:00:00
33 45 39
2022-06-30 00:00:00
34 46 40
2022-05-31 00:00:00
35 47 41
2022-04-30 00:00:

In [ ]:
temp = completed_df.reset_index()

In [ ]:
temp[temp['index']==0]

,index,date,call_to_branch_std_6m,call_to_branch_std_12m,visit_to_branch_std_6m,visit_to_branch_std_12m,total_to_branch_std_6m,total_to_branch_std_12m,so_eng_1m_ago,so_eng_3m_ago,so_eng_1m_ago_count,so_eng_3m_ago_count,small_bus_owner_flag,std_prod_own_flag,tfsa_prod_own_12m,tranact_prd_count,unsecure_loc_count_flag
0,0,2025-02-28 00:00:00,1.095445,1.087115,0.816497,1.969464,2.443296,2.244155,1,1,54.0,47.0,0,0,1,0,1
10000,0,2025-01-31 00:00:00,1.095445,1.083625,0.816497,1.969464,2.443296,2.274225,1,1,54.0,47.0,1,0,1,0,1
20000,0,2024-12-31 00:00:00,1.169045,1.073087,0.816497,1.969464,2.539088,2.303117,1,1,54.0,47.0,1,1,1,0,1
30000,0,2024-11-30 00:00:00,1.366260,1.073087,0.816497,1.969464,2.662876,2.303117,1,1,54.0,47.0,0,1,0,0,1
40000,0,2024-10-31 00:00:00,0.816497,0.668558,0.816497,1.969464,2.724746,2.327763,1,1,54.0,47.0,1,1,1,0,0
50000,0,2024-09-30 00:00:00,0.752773,0.668558,0.816497,1.969464,2.800162,2.327763,1,1,54.0,47.0,0,0,0,0,0
60000,0,2024-08-31 00:00:00,0.894427,0.668558,0.816497,1.969464,2.902455,2.327763,1,1,54.0,47.0,1,0,1,1,0
70000,0,2024-07-31 00:00:00,0.752773,0.668558,0.816497,1.969464,2.968267,2.327763,1,1,54.0,47.0,0,1,1,1,1
80000,0,2024-06-30 00:00:00,0.752773,0.866025,0.816497,1.969464,2.968267,2.293264,1,1,54.0,47.0,1,0,0,0,1
90000,0,2024-05-31 00:00:00,0.632456,1.083625,0.816497,1.969464,2.870962,2.212858,1,1,54.0,47.0,0,0,0,0,1


In [ ]:
df

,calls_to_branch,visit_to_branch,so_eng,calls_to_branch_m_1,visit_to_branch_m_1,so_eng_m_1_count,calls_to_branch_m_2,visit_to_branch_m_2,so_eng_m_2_count,calls_to_branch_m_3,...,so_eng_m_56_count,calls_to_branch_m_57,visit_to_branch_m_57,so_eng_m_57_count,calls_to_branch_m_58,visit_to_branch_m_58,so_eng_m_58_count,calls_to_branch_m_59,visit_to_branch_m_59,so_eng_m_59_count
0,3.0,7.0,51.0,3.0,8.0,54.0,3.0,8.0,47.0,2.0,...,89.0,7.0,11.0,89.0,8.0,11.0,90.0,10.0,11.0,89.0
1,2.0,6.0,-0.0,2.0,7.0,1.0,2.0,7.0,0.0,2.0,...,22.0,0.0,14.0,22.0,1.0,13.0,13.0,2.0,12.0,9.0
2,3.0,6.0,-7.0,3.0,6.0,0.0,3.0,5.0,2.0,3.0,...,47.0,13.0,9.0,51.0,13.0,8.0,52.0,14.0,7.0,52.0
3,3.0,4.0,51.0,3.0,4.0,47.0,2.0,5.0,51.0,1.0,...,66.0,2.0,10.0,68.0,3.0,9.0,75.0,2.0,10.0,72.0
4,2.0,7.0,40.0,1.0,6.0,42.0,1.0,6.0,39.0,1.0,...,17.0,3.0,17.0,22.0,5.0,17.0,21.0,6.0,17.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4.0,3.0,74.0,4.0,3.0,79.0,3.0,3.0,79.0,3.0,...,103.0,14.0,10.0,100.0,16.0,11.0,105.0,18.0,11.0,107.0
9996,3.0,1.0,50.0,2.0,0.0,47.0,2.0,1.0,54.0,2.0,...,136.0,9.0,10.0,144.0,9.0,9.0,142.0,10.0,8.0,147.0
9997,4.0,7.0,39.0,5.0,7.0,45.0,3.0,7.0,53.0,4.0,...,61.0,4.0,5.0,63.0,5.0,7.0,71.0,5.0,6.0,76.0
9998,4.0,3.0,22.0,5.0,3.0,17.0,4.0,3.0,16.0,4.0,...,53.0,1.0,5.0,50.0,1.0,4.0,56.0,1.0,3.0,55.0
